In [33]:
## compute entropy of spectrum signals as biomarkers
## udpates on Apr 11 2022:
## save spectrum as time points by frequency for each subject.
## calculate spectrum only on test set.

import glob
import pandas as pd
import numpy as np
import re
import os

## read harmonics and time signals:

## hcp data:
# input_harmonics_wavelets = 'hcp_out02_harmonics_100'
# input_time_signal = '/home/xin/Downloads/FullData_Oct26/Scan1/'
# output_dir = 'hcp_out03_spectrum_power'

## adni data:
input_harmonics_wavelets = 'adni_out02_harmonics'
input_time_signal = '/home/xin/Downloads/BrainImaging_UNC/out04_adni_roi_signals1'
output_dir = 'adni_out03_spectrum_power'

## Notes:
# got NaN values in wavelets (e.g for ROI 3 all values are nan), 
# but time signals does not have nans...
# need to check where those NaNs are derived...

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

harmonic_wavelets_files = glob.glob(input_harmonics_wavelets + '/CommonHarWavelets_Harmonics*.csv')
harmonic_wavelets_files.sort()

## ADNI:
time_signal_files = pd.read_csv('/home/xin/Downloads/Harmonics/adni_out01_time_signal_test_train_split/test_set_roi_signals.csv', header = None)    
time_signal_files = time_signal_files.iloc[:,0].values.tolist()
## HCP data:
# time_signal_files = glob.glob(input_time_signal + '/TimeSeries*.csv')


In [26]:
# for each wavelets (corresponding to a brain region) calcluate inner product with roi signals for each subject:

spectrum_list = [] # nested array with row: brain region and column: subject.
brain_region_list = []
subject_id_list = []

fill_subject_id = True 
test_break = False

for file_wavelets in harmonic_wavelets_files:
    print(file_wavelets)
    spectrum_sub_list = []
    wavelets = pd.read_csv(file_wavelets, header = None)
    
    if wavelets.isna().any(axis = None):
        print('Nan values for wavelets:' + file_wavelets)
        spectrum_list.append(spectrum_sub_list)
        continue
    
    brain_region = file_wavelets[-7:-4]
    brain_region_list.append(brain_region)
    
    for file_time_signal in time_signal_files:
        
        ## adni:
        subject_id = re.search('roi_signals_power264_(.*).txt', file_time_signal).group(1)
        time_signal = pd.read_csv(file_time_signal, header = 0, sep = '\t')
        time_signal.drop(columns = ['Unnamed: 264'], inplace = True)
        time_signal.columns = time_signal.columns.astype(int)
        time_signal = time_signal.sort_index(ascending=True, axis=1)
        
        ## global signal regression:
        time_signal = time_signal.sub(time_signal.mean(axis = 1), axis = 0)
        
        ## hcp:
        # subject_id = re.search('TimeSeries(.*).csv', file_time_signal).group(1)
        # time_signal = pd.read_csv(file_time_signal, header = None, sep = ',')
        # time_signal = time_signal.T
        
        # print(subject_id)

        # check nan values:
        if time_signal.isna().any(axis = None):
            print('Nan value for time_signal:' + subject_id)
            spectrum_sub_list.append([])
            # test_break = True
            continue

        # we could use this to directly skip files with NaN values. not implemented yet...
        if fill_subject_id:
            subject_id_list.append(subject_id)
        
        ## compute inner product for each time point:
        P = np.matmul(time_signal.values, wavelets.values)
        spectrum_sub_list.append(P)
    
#     if test_break:
#         break
        
    fill_subject_id = False
    spectrum_list.append(spectrum_sub_list)
        

adni_out02_harmonics/CommonHarWavelets_Harmonics001.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics002.csv
Nan values for wavelets:adni_out02_harmonics/CommonHarWavelets_Harmonics002.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics003.csv
Nan values for wavelets:adni_out02_harmonics/CommonHarWavelets_Harmonics003.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics004.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics005.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics006.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics007.csv
Nan values for wavelets:adni_out02_harmonics/CommonHarWavelets_Harmonics007.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics008.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics009.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics010.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics011.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics012.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics013.csv
adni_out02_harmonics/CommonHarWa

In [27]:
time_signal

,1,2,3,4,5,6,7,8,9,10,...,255,256,257,258,259,260,261,262,263,264
0,518.242326,-9.174340,-54.931916,-145.952118,-212.841007,212.908993,503.936771,172.068084,-273.841007,37.158993,...,-469.533315,-321.568280,211.575660,458.795357,110.958993,531.381215,213.466685,628.082070,-6.063229,86.058993
1,517.376338,-25.290329,-50.108511,-145.734774,-215.290329,216.772171,505.709671,173.891489,-276.653966,33.543004,...,-439.828791,-314.472147,214.209671,456.800580,110.209671,532.598560,217.248132,630.171209,-4.401440,86.209671
2,522.765977,-32.650690,-48.877962,-147.372912,-213.750690,220.224310,495.738199,174.576583,-277.332508,33.182644,...,-442.227613,-316.423417,209.849310,461.758401,110.049310,537.849310,215.464695,639.157002,-3.928467,90.249310
3,521.260510,-16.822823,-50.641005,-151.156157,-212.556157,222.927177,504.843844,177.722631,-276.641005,32.843843,...,-450.822823,-326.550096,206.843843,460.904450,111.177177,538.177177,225.023331,636.638715,-8.600601,87.477177
4,521.899688,-13.850313,-54.653343,-149.016979,-216.283646,225.420521,507.316354,179.528475,-278.107889,37.983021,...,-443.555441,-326.016979,212.733021,445.710294,109.583021,544.649688,226.906097,626.290713,-3.683646,87.683021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,475.369089,-31.297578,-52.555154,-142.242022,-205.797578,195.223256,462.869089,164.626665,-267.827881,66.869089,...,-483.464244,-310.100608,199.785756,424.717574,119.535756,518.535756,203.381909,620.843448,-0.464244,85.335756
193,492.985429,-47.347904,-61.514571,-153.181238,-199.114571,199.172929,465.485429,166.576338,-270.514571,71.485429,...,-490.053032,-315.605480,181.068762,437.576338,123.685429,493.152096,192.562352,619.793121,-3.736793,92.085429
194,542.725292,-82.024708,-69.297436,-162.358042,-213.091375,200.600292,488.530848,170.884383,-271.570163,60.308625,...,-454.409324,-330.842890,187.725292,451.157110,141.375292,511.530848,194.898369,650.821446,-18.135819,100.375292
195,495.109298,-37.474035,-61.019490,-158.696257,-200.540702,208.900965,480.525965,168.798692,-273.019490,72.525965,...,-464.550958,-338.019490,190.942632,448.435056,122.125965,513.748187,197.910580,610.525965,-10.251813,83.825965


In [28]:
# check for NaN values in wavelets:
print(wavelets.isna().any(axis=0))

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool


In [35]:
## save spectrum for each subject and brain region:
        
for brain_index in range(len(spectrum_list)):
    if len(spectrum_list[brain_index])==0:
        continue
    for subj in range(len(spectrum_list[0])):
        P = spectrum_list[brain_index][subj]

        out_file_name = output_dir + '/spectrum_subj' + '{:03}'.format(subj) + '_roi_' + '{:03}'.format(brain_index) + '.csv'
        np.savetxt(out_file_name, np.array(P), delimiter = ',')


In [34]:
len(spectrum_list)

264

In [ ]:
## make histogram

## calculate entropy